<a href="https://colab.research.google.com/github/shahpriyankaj/iedgar_sec_filing_analysis/blob/main/edgar_genai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Task #2 – Gen AI

In [1]:
# Installation steps for PySpark on Google Collab
!apt-get update # Update apt-get repository.
!apt-get install openjdk-8-jdk-headless -qq > /dev/null # Install Java.
!wget -q http://archive.apache.org/dist/spark/spark-3.4.4/spark-3.4.4-bin-hadoop3.tgz # Download Apache Sparks.
!tar xf spark-3.4.4-bin-hadoop3.tgz # Unzip the tgz file.
!pip install -q findspark # Install findspark. Adds PySpark to the System path during runtime.

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [69.9 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,378 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3,813 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,679 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,037 kB]
Get:13 http://security.u

In [11]:
#Set environment variables to use PySpark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.4-bin-hadoop3"

# Create spark-defaults.conf and set driver memory
!echo 'spark.driver.memory 50g' > /content/spark-3.4.4-bin-hadoop3/conf/spark-defaults.conf


# Initialize findspark
import findspark
findspark.init()

# Create a PySpark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

In [3]:

# Installing datasets library to download the data from Huggingface
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.9 MB/s eta 0:00:00


In [4]:
# Step 2: Load datasets of Year 2018, 2019 and 2020 from https://huggingface.co/datasets/eloukas/edgar-corpus. Below code was mentioned on Huggingface to download yearly training dataset.
import datasets
import pandas as pd

years = ["year_2018", "year_2019", "year_2020"]
dfs = []

for y in years:
    # Only using train dataset for this task.
    ds = datasets.load_dataset("eloukas/edgar-corpus", y, split="train", trust_remote_code=True)
    dfs.append(ds.to_pandas())

combined_df = pd.concat(dfs, ignore_index=True)

README.md:   0%|          | 0.00/43.7k [00:00<?, ?B/s]

edgar-corpus.py:   0%|          | 0.00/4.64k [00:00<?, ?B/s]

train.jsonl:   0%|          | 0.00/1.45G [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/184M [00:00<?, ?B/s]

validate.jsonl:   0%|          | 0.00/183M [00:00<?, ?B/s]

...

test.jsonl:   0%|          | 0.00/195M [00:00<?, ?B/s]

validate.jsonl:   0%|          | 0.00/195M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5480 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/686 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/685 [00:00<?, ? examples/s]

In [12]:
#Step 2: Convert to PySpark and Filter 1 Company
'''
Below code gave me out of memory error even after givine the Spark driver memory to 50g. Hence, I worked with pandas to filter single company.
from pyspark.sql.functions import col
spark_df = spark.createDataFrame(combined_df)

# Choose a company based on highest count
selected_cik = spark_df.groupBy("cik").count().orderBy("count", ascending=False).first()['cik']
company_df = spark_df.filter(col("cik") == selected_cik)
'''

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.readRDDFromFile.
: java.lang.OutOfMemoryError: Java heap space


In [21]:
#Step 2: Filter 1 Company
selected_cik = combined_df.groupby("cik").size().sort_values(ascending=False).index[0]
company_df = combined_df[combined_df["cik"] == selected_cik

In [64]:
selected_cik

'1001907'

In [22]:
# Step 3: Chunk Text Using LangChain + PySpark UDF. Other chunking strategies can be tried here such as semantic chunking, markdown splitting, etc.
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pyspark.sql.functions import udf, struct
from pyspark.sql.types import ArrayType, StringType

splitter = RecursiveCharacterTextSplitter(
            chunk_size=512,
            chunk_overlap=50,
            separators=["\n", "\n\n"]
        )

def split_all_sections(row):
    text = "\n".join([str(row[sec]) for sec in row.asDict() if sec.startswith('section_') and row[sec]])
    return splitter.split_text(text)

# Convert company_df to PySpark DataFrame
company_df = spark.createDataFrame(company_df) # Convert to PySpark DataFrame
split_udf = udf(split_all_sections, ArrayType(StringType()))
# Pass individual columns to the UDF using struct
chunked_df = company_df.withColumn("chunks", split_udf(struct([company_df[x] for x in company_df.columns])))
#chunked_df = company_df.withColumn("chunks", split_udf(company_df))

In [23]:
#Step 4: Generate Embeddings using all-MiniLM-L6-v2 model for Each Chunk
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer("all-MiniLM-L6-v2")

# Convert chunks to flat list
chunk_rows = chunked_df.select("year", "chunks").collect()

all_chunks = []
for row in chunk_rows:
    year = row['year']
    for chunk in row['chunks']:
        all_chunks.append({"year": year, "text": chunk})

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [24]:
# Create embeddings
for chunk in all_chunks:
    chunk["embedding"] = model.encode(chunk["text"])

In [ ]:
# Install relevant packages for loading the llama model
!pip install llama-cpp-python
!pip install huggingface-hub>=0.17.1

#Mount the google drive to store the model
from google.colab import drive
drive.mount('/content/drive')

!huggingface-cli login #Add the token from Hugging face account

# Download the model into the drive
!huggingface-cli download TheBloke/Llama-2-7B-Chat-GGUF llama-2-7b-chat.Q4_K_M.gguf \
--local-dir /content/drive/MyDrive/models/llama2 \
--local-dir-use-symlinks False

In [ ]:
# Load the Llama model
from llama_cpp import Llama

llm = Llama(model_path="/content/drive/MyDrive/models/llama2/llama-2-7b-chat.Q4_K_M.gguf")

In [55]:
'''
Step 5: Create Query + Prompt to extract the Specific Year.
Why Use LLM for Year Extraction?
- Handles different phrasing ("in 2020", "during 2019", "FY18")
- Avoids brittle regex rules
- Makes the system more adaptable to natural language
Regex can be used to extract the year, but LLM is the scalable approach.
'''

def extract_year_from_query(query):
  prompt = f"""
  Extract the year mentioned in the following query.
  If no year is mentioned, return "None", If multiple years are mentioned, return only first year.
  Query: "{query}"
  """

  output = llm(prompt, max_tokens=10)
  year = output["choices"][0]["text"].strip().split()[-1]
  print(year)  # Expected: 2020
  return year

In [61]:
'''
Step 6: Get top 5 chunks for a query and create a Prompt to generate the answer.
Used cosine similarity to retrieve the top 5 chunks from the specific year for a query.
Use prompt engineering to create the prompt to include system message, contect i.e. top 5 chunks and a query.
Use LLM to generate the response using the context.
Please note, the prompt is created based on my understanding that this task will be used as QnA chatbot.
'''
from sklearn.metrics.pairwise import cosine_similarity
from langchain import PromptTemplate

def get_top_5_chunks(query):
  query_embedding = model.encode(query)
  year = extract_year_from_query(query)
  if year is None: #Selecting defualt as 2020 in case of error. If LLM is not able to extract, we can use regex to extract it.
    year = '2020'
  # Retrieve top 5 chunks from year extracted using LLM
  year_chunks = [c for c in all_chunks if c["year"] == year]
  similarities = cosine_similarity([query_embedding], [c["embedding"] for c in year_chunks])[0]

  top_k = 5
  top_indices = similarities.argsort()[-top_k:][::-1]
  top_chunks = [year_chunks[i]['text'] for i in top_indices]
  context = "\n".join(chunk for chunk in top_chunks)
  return context

def create_prompt(query, context):
  # Create the Prompt for LLM to generate the answer from retrieved chunks
  # System message for LLM prompt
  system_message = '''
            You are a helpful assistant for question-answering tasks. Provide an answer concisely based on the context only, without using general knowledge.
            Please correct any grammatical errors for improved readability.
            If the context does not contain relevant information to answer the question, state that the answer is not available in the given context.
            '''
  #Generate Prompt
  prompt = '''
          [INST] <<SYS>>{system_message}<</SYS>>
          Context: {context}
          Question: {query} [/INST]
          '''
  # Create a prompt template
  prompt_template = PromptTemplate(input_variables=["system_message", "context", "query"], template=prompt)

  # Format the final prompt with the query and search results
  final_prompt = prompt_template.format(system_message = system_message, query=query, context=context[:1024])
  return final_prompt

def get_response(prompt):
  response = llm(
              prompt=final_prompt,
              max_tokens=4000,
              temperature=0.5,
              top_p=0.9,
              repeat_penalty=1.2,
              top_k=20,
              echo=False
          )
  answer = response["choices"][0]["text"]
  return answer

In [59]:
query = "What are the key risk factors for the company in 2020?"
context = get_top_5_chunks(query)
prompt = create_prompt(query, context)
answer = get_response(prompt)
print(answer)

Llama.generate: 3 prefix-match hit, remaining 57 prompt tokens to eval
llama_perf_context_print:        load time =    2949.18 ms
llama_perf_context_print: prompt eval time =    4946.53 ms /    57 tokens (   86.78 ms per token,    11.52 tokens per second)
llama_perf_context_print:        eval time =    1082.42 ms /     9 runs   (  120.27 ms per token,     8.31 tokens per second)
llama_perf_context_print:       total time =    6033.64 ms /    66 tokens
Llama.generate: 3 prefix-match hit, remaining 337 prompt tokens to eval


2020


llama_perf_context_print:        load time =    2949.18 ms
llama_perf_context_print: prompt eval time =   21198.14 ms /   337 tokens (   62.90 ms per token,    15.90 tokens per second)
llama_perf_context_print:        eval time =   17220.38 ms /   142 runs   (  121.27 ms per token,     8.25 tokens per second)
llama_perf_context_print:       total time =   38574.83 ms /   479 tokens


 Based on the given context, it is not possible to determine the key risk factors for the company in 2020 with certainty. The context highlights that product development involves a high degree of risk and uncertainty, and the COVID-19 pandemic has introduced additional risks and uncertainties. As a result, the extent of the impact of the COVID-19 pandemic on the company's business is highly uncertain and difficult to predict. Therefore, it cannot be determined what the key risk factors are for the company in 2020 based solely on the given context.
If you have any other questions or need further assistance, please feel free to ask!


In [67]:

  '''
  # Step 7: Create Validation Dataset
  Below golden questions and answers are generated by chatGPT by providing section 1 of the 2020 for company 1001907.
  Ideally, the golden examples should be provided by Business and can be stored in persistent storage for evaluation.
  For this task, I am manually storing them in pandas df.
  '''
data = {
    "questions": [
        "What subsidiary did Astrotech launch in 2020 related to breath analysis?",
        "What financial arrangement did Astrotech enter into in February 2020?",
        #"Which product was developed by BreathTech in 2020 to detect COVID-19?",
        #"How much did Astrotech raise from the stock offering in March 2020?",
        #"What type of loan did Astrotech secure in April 2020 under the CARES Act?"
    ],
    "golden_responses": [
        "BreathTech was launched in 2020 to develop breath analysis tools.",
        "Astrotech issued a $1M secured promissory note to Mr. Pickens in Feb 2020.",
        #"The BreathTest-1000 was developed by BreathTech in 2020 to detect COVID-19.",
        #"Astrotech raised $1.6M and $2.9M from two stock offerings in March 2020.",
        #"Astrotech secured a $542K PPP loan in April 2020 under the CARES Act."
      ]
}
golden_examples = pd.DataFrame(data)

In [ ]:
!pip install rouge_score

In [73]:
'''
Task: Demonstrate that your LLM can retrieve the correct chunks from your embedding object for the correct year
We can use evaluation metrics to compare the LLM-generated response with golden examples to demonstrate that the LLM is retrieving chunks from correct year.
Below code is first extracting the year from a query using LLM, then getting top 5 chunks using cosine similarity and use these chunks as context to generate response using LLM.
The generated response is then compared with golden responses. These golden responses are created from year 2020 from section 1.
There are other evaluation metrics such as Bleu, BERTScore, etc. can be used to check the performance of LLM.
'''

from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

correct = 0
all_scores = []
for idx, val in golden_examples.iterrows():
    context = get_top_5_chunks(val["questions"]) #This function will retrieve year from query using LLM and retrieve top 5 similar chunks from that year for a query
    prompt = create_prompt(val["questions"], context) #This function will create prompt using context
    answer = get_response(prompt) #This function will generate the response using LLM
    print(answer)
    scores = scorer.score(val["golden_responses"].lower(), answer.lower())
    all_scores.append(scores)
all_scores

Llama.generate: 3 prefix-match hit, remaining 61 prompt tokens to eval
llama_perf_context_print:        load time =    2949.18 ms
llama_perf_context_print: prompt eval time =    5241.02 ms /    61 tokens (   85.92 ms per token,    11.64 tokens per second)
llama_perf_context_print:        eval time =    1058.21 ms /     9 runs   (  117.58 ms per token,     8.50 tokens per second)
llama_perf_context_print:       total time =    6303.80 ms /    70 tokens
Llama.generate: 3 prefix-match hit, remaining 337 prompt tokens to eval


2020


llama_perf_context_print:        load time =    2949.18 ms
llama_perf_context_print: prompt eval time =   21793.74 ms /   337 tokens (   64.67 ms per token,    15.46 tokens per second)
llama_perf_context_print:        eval time =   21014.06 ms /   171 runs   (  122.89 ms per token,     8.14 tokens per second)
llama_perf_context_print:       total time =   42999.58 ms /   508 tokens
Llama.generate: 3 prefix-match hit, remaining 58 prompt tokens to eval


 Based on the given context, the key risk factors for the company in 2020 are:
1. The COVID-19 pandemic: As a global health crisis, the COVID-19 pandemic poses significant risks to the Company's business operations and financial performance. These include supply chain disruptions, reduced consumer spending, increased costs associated with remote work arrangements, and potential regulatory changes aimed at slowing the spread of the virus.
2. Product development risk: The company faces a high degree of uncertainty in developing new products that may not be successfully developed or commercially successful. This includes risks related to product design, testing, market acceptance, and competition from other companies. 
3. Market authorization risk: The Company must obtain full market authorization for its products before they can be


llama_perf_context_print:        load time =    2949.18 ms
llama_perf_context_print: prompt eval time =    3799.46 ms /    58 tokens (   65.51 ms per token,    15.27 tokens per second)
llama_perf_context_print:        eval time =    1066.96 ms /     9 runs   (  118.55 ms per token,     8.44 tokens per second)
llama_perf_context_print:       total time =    4870.88 ms /    67 tokens
Llama.generate: 3 prefix-match hit, remaining 337 prompt tokens to eval


2020


llama_perf_context_print:        load time =    2949.18 ms
llama_perf_context_print: prompt eval time =   23050.99 ms /   337 tokens (   68.40 ms per token,    14.62 tokens per second)
llama_perf_context_print:        eval time =   17524.97 ms /   143 runs   (  122.55 ms per token,     8.16 tokens per second)
llama_perf_context_print:       total time =   40736.76 ms /   480 tokens


 Based on the context provided, there are several key risk factors for the company in 2020:
1. The COVID-19 pandemic and its impact on product development and commercialization, including increased risks and uncertainties related to developing and launching new products and services during a global health crisis.
2. Uncertainty regarding the effectiveness of responses taken by the company, other businesses, and governments in mitigating the COVID-19 pandemic's impact on its operations and financial performance. 
3. The potential for negative economic impacts resulting from the pandemic, including disruptions to capital markets and economies worldwide.


[{'rouge1': Score(precision=0.031746031746031744, recall=0.4, fmeasure=0.058823529411764705),
  'rouge2': Score(precision=0.008, recall=0.1111111111111111, fmeasure=0.014925373134328356),
  'rougeL': Score(precision=0.031746031746031744, recall=0.4, fmeasure=0.058823529411764705)},
 {'rouge1': Score(precision=0.041666666666666664, recall=0.3076923076923077, fmeasure=0.07339449541284403),
  'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0),
  'rougeL': Score(precision=0.020833333333333332, recall=0.15384615384615385, fmeasure=0.03669724770642201)}]